# JF image read
# Direct Jungfrau data download

In [122]:
%matplotlib notebook
import os
import numpy as np
import scipy
from scipy import optimize, ndimage

import matplotlib.pyplot as plt
from matplotlib import cm
from extra_data import *
from extra_data import by_id
import pandas as pd
import zipfile

## Prepare folder locations with experimental run and proposal number
## Change this cell!

In [123]:
# Create proposal directory
#propno = "3213" #"p002353" 
propno = "2838"
home_dir = "/home/jchal/"
prop_dir = home_dir+"p"+propno

os.chdir(home_dir)

# Make the output directory - caution, this makes the directory here where you run this script.
if not os.path.exists("p"+propno):
    os.mkdir("p"+propno)
    os.chdir(home_dir+"p"+propno)
else:
    os.chdir(home_dir+"p"+propno)

print(os.getcwd())

/home/jchal/p2838


### Open run and create run folder

In [109]:
#Open run and create run folder
run_number = 35
run_dir = prop_dir + "/"+"Run"+str(run_number)

run = open_run(propno, run_number, "proc")
#run = open_run(propno, 302, "raw")
#run.info()

os.chdir(prop_dir)

if not os.path.exists("Run"+str(run_number)):
    os.mkdir("Run"+str(run_number))
    os.chdir(run_dir)    
else:
    os.chdir(run_dir)
   
print("Current directory: " + os.getcwd())
print("Run " + str(run_number) + " contains " + str(len(run.train_ids)) + " train_IDs.")

Current directory: /home/jchal/p2838/Run35
Run 35 contains 10964 train_IDs.


In [65]:
run.instrument_sources
#run.info()

frozenset({'HED_IA1_EPX100-1/DET/RECEIVER:daqOutput',
           'HED_IA1_EPX100-2/DET/RECEIVER:daqOutput',
           'HED_IA1_JF500K1/DET/JNGFR01:daqOutput',
           'HED_IA1_JF500K2/DET/JNGFR02:daqOutput',
           'HED_IA1_JF500K3/DET/JNGFR03:daqOutput',
           'HED_IA1_TSC-1/CAM/TARGET_FRONT:daqOutput',
           'HED_IA1_TSC-2/CAM/TARGET_BACK:daqOutput',
           'HED_OPT_IPM/ADC/1:channel_0.output',
           'HED_OPT_IPM/ADC/1:channel_1.output',
           'HED_OPT_IPM/ADC/1:channel_2.output',
           'HED_OPT_IPM/ADC/1:channel_3.output',
           'HED_OPT_IPM/ADC/1:channel_4.output',
           'HED_OPT_IPM/ADC/1:channel_5.output',
           'HED_OPT_IPM/ADC/1:channel_6.output',
           'HED_OPT_IPM/ADC/1:channel_7.output',
           'HED_OPT_IPM/ADC/1:channel_8.output',
           'HED_OPT_IPM/ADC/1:channel_9.output',
           'HED_RR_SYS/MDL/KARABACON:output',
           'HED_XTD6_XGM/XGM/DOOCS:output',
           'SA2_XTD1_XGM/XGM/DOOCS:output',
   

In [110]:
### Jungfrau data extraction and zipping

# JF3 = run['HED_IA1_JF500K3/DET/JNGFR03:daqOutput', 'data.adc'].xarray().squeeze().mean("trainId")
# this averages over all train IDs and squeezes out the extra nested array in JF images
#help(run)

os.chdir(run_dir)

#N_trains = len(run.train_ids)
N_trains = 200
start = 0
print("Extracting files ...")

for i in range(start, start+N_trains): 
    sel = run.select_trains(i)
    JF3 = sel['HED_IA1_JF500K3/DET/JNGFR03:daqOutput', 'data.adc'].xarray().squeeze()
    #print("Extracted train_ID " + str(run.train_ids[i]))
    
    # Export to unformated file
    f = open("T"+str(run.train_ids[i])+".dat", "w")
    JF3.data.tofile(f)
    f.close()
    
    #Export to TIF file
    #plt.imsave("T"+str(run.train_ids[i])+".tif",JF3.data, format="tif", cmap = cm.gray)
    #print(str(i) + "," + str(JF3.data.max()))   
    

# ZIP the data in the prop folder
os.chdir(prop_dir)
print("Saving ZIP file to: " +os.getcwd())
#print(os.listdir('.'))

zf = zipfile.ZipFile("Run"+str(run_number)+".zip", "w")
for file in os.listdir("Run"+str(run_number)):
    #print(file)
    #zf.write(os.path.join("Run"+str(run_number)+"/", file))
    zf.write(os.path.join("Run"+str(run_number), file))
    
zf.close()


Extracting files ...
Saving ZIP file to: /home/jchal/p2838


In [124]:
# Delete all files in the run directory!!!
run_number = 36
run_dir = prop_dir + "/"+"Run"+str(run_number)
#run_dir = prop_dir

os.chdir(run_dir)
print("Deleting files in: " + os.getcwd())
for file in os.listdir("."):    
    if os.path.isfile(file):
        os.remove(file)
        #print("Delete: " + file)

Deleting files in: /home/jchal/p2838
